### THIS IS ME TRYING OUT ###

In [11]:
!pip install transformers
!pip install sentencepiece

!pip install sacrebleu

     |████████████████████████████████| 106 kB 9.3 MB/s            
     |████████████████████████████████| 8.8 MB 18.1 MB/s            


In [6]:
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the validation and testing datasets
validation_dataset = load_dataset('wmt16', 'de-en', split='validation')
test_dataset = load_dataset('wmt16', 'de-en', split='test')

# Load the t5-small model and tokenizer
model_name = 't5-small'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Function to generate translations
def generate_translation(batch):
    inputs = tokenizer('translation German to English: ' + batch['translation']['de'], return_tensors='pt', truncation=True, padding=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch['translation']['en'], return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs, labels=labels.input_ids)
    return outputs.loss, outputs.logits

# Generate translations for validation set
validation_results = validation_dataset.map(generate_translation, batched=True, batch_size=4)

# Generate translations for test set
test_results = test_dataset.map(generate_translation, batched=True, batch_size=4)

# Calculate evaluation metrics
from datasets import load_metric

metric = load_metric('sacrebleu')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return metric.compute(predictions=predictions, references=labels)

validation_metric = validation_results['translation'].map(compute_metrics)
test_metric = test_results['translation'].map(compute_metrics)

print('Validation BLEU Score:', validation_metric['score'])
print('Test BLEU Score:', test_metric['score'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/2169 [00:00<?, ? examples/s]

TypeError: list indices must be integers or slices, not str

In [1]:
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the validation and testing datasets
validation_dataset = load_dataset('wmt16', 'de-en', split='validation')
test_dataset = load_dataset('wmt16', 'de-en', split='test')

# Load the t5-small model and tokenizer
model_name = 't5-small'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

  # Function to generate translations
task_prefix = "translate German to English: "

def generate_translation(batch):
    inputs = tokenizer([task_prefix + entry['de'] for entry in batch['translation']], return_tensors='pt', padding=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer([task_prefix + entry['en'] for entry in batch['translation']], return_tensors='pt', padding=True)
    outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], 
                            labels=labels["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return {"translations": preds}

# Generate translations for validation set
validation_results = validation_dataset.map(generate_translation, batched=True, batch_size=4)

# Generate translations for test set
test_results = test_dataset.map(generate_translation, batched=True, batch_size=4)



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/2169 [00:00<?, ? examples/s]

/opt/homebrew/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2999 [00:00<?, ? examples/s]

/var/folders/f5/sy3261ln1m3518y0gc0sptqr0000gn/T/ipykernel_81792/478642144.py:34: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('sacrebleu')
/opt/homebrew/lib/python3.11/site-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


AttributeError: 'list' object has no attribute 'map'

In [16]:
print(validation_results)
print(validation_results['translation'])
print(validation_results['translations'][0])
print(validation_results['translations'])

Dataset({
    features: ['translation', 'translations'],
    num_rows: 2169
})
[{'de': 'Die Premierminister Indiens und Japans trafen sich in Tokio.', 'en': 'India and Japan prime ministers meet in Tokyo'}, {'de': 'Indiens neuer Premierminister Narendra Modi trifft bei seinem ersten wichtigen Auslandsbesuch seit seinem Wahlsieg im Mai seinen japanischen Amtskollegen Shinzo Abe in Toko, um wirtschaftliche und sicherheitspolitische Beziehungen zu besprechen.', 'en': "India's new prime minister, Narendra Modi, is meeting his Japanese counterpart, Shinzo Abe, in Tokyo to discuss economic and security ties, on his first major foreign visit since winning May's election."}, {'de': 'Herr Modi befindet sich auf einer fünftägigen Reise nach Japan, um die wirtschaftlichen Beziehungen mit der drittgrößten Wirtschaftsnation der Welt zu festigen.', 'en': 'Mr Modi is on a five-day trip to Japan to strengthen economic ties with the third largest economy in the world.'}, {'de': 'Pläne für eine stärkere

In [9]:
# Calculate evaluation metrics
from datasets import load_metric

metric = load_metric('sacrebleu')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return metric.compute(predictions=predictions, references=labels)

validation_metric = (validation_results['translation']).map(compute_metrics)
test_metric = (test_results['translation']).map(compute_metrics)

print('Validation BLEU Score:', validation_metric['score'])
print('Test BLEU Score:', test_metric['score'])


/opt/homebrew/lib/python3.11/site-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


AttributeError: 'list' object has no attribute 'map'

In [13]:
from datasets import load_metric

# Load the BLEU metric
bleu = evaluate.load("bleu")

# Define a function to compute metrics
def compute_metrics(eval_pred):
 
# Access the translation results from the validation_results dictionary
translation_results = validation_results['translation']

# Iterate over each translation result and compute metrics
validation_metric = []
for result in translation_results:
    metric_result = compute_metrics(result)
    validation_metric.append(metric_result)


/opt/homebrew/lib/python3.11/site-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


TypeError: string indices must be integers, not 'str'